In [20]:
import os

import torch
from torchvision import transforms

import pandas as pd
import numpy as np
import random

from torchvision.datasets import CIFAR10
from brain import Agent

from tqdm import tqdm

device = "cuda"

In [2]:
def set_seeds(seed):
    torch.manual_seed(seed)  # Sets seed for PyTorch RNG
    torch.cuda.manual_seed_all(seed)  # Sets seeds of GPU RNG
    np.random.seed(seed=seed)  # Set seed for NumPy RNG
    random.seed(seed)  # Set seed for random RNG

set_seeds(123)


In [12]:
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
 ])

val_transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
 ])

batch_size=64

trainset = CIFAR10("../data", train=True, transform=train_transform, download=True)
train_loader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=batch_size)

valset = CIFAR10("../data", train=False, transform=val_transform, download=True)
val_loader = torch.utils.data.DataLoader(valset, shuffle=True, batch_size=batch_size)


0it [00:00, ?it/s]

Files already downloaded and verified


In [4]:
model = Agent().to(device)

loss_fn = torch.nn.CrossEntropyLoss()
lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)#, momentum=0.9)


2433388352


In [5]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter("tensorboard")
dataiter = iter(train_loader)
images, labels = dataiter.next()
writer.add_graph(model, images.to(device))
writer.flush()


/home/intern_1/Desktop/Dynamic-ResNet/brain.py:45: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  batch_size = int(c1.shape[0]/64)


In [6]:
def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    size = len(dataloader.dataset)
    trainloss = 0
    train_correct = 0
    train_total = 0
    for batch, (X, y) in enumerate(tqdm(dataloader)):
        model = model.train()
        X, y = X.to(device), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        trainloss += loss.item()
        
        pred = torch.argmax(pred, dim=1)
        train_total += len(pred)
        train_correct += (pred == y).sum().item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    
    val_correct = 0
    val_total = 0
    valloss = 0

    model = model.eval()
    with torch.no_grad():
        for i, (X,y) in enumerate(val_loader):
            X, y = X.to(device), y.to(device)
            pred = model(X)
            valloss += loss_fn(pred, y).item()
            pred = torch.argmax(pred, dim=1)
            val_total += len(pred)
            val_correct += (pred == y).sum().item()

    print(f"Epoch: {epoch}; Train Loss: {trainloss/train_total}, Val Loss: {valloss/val_total}")

    writer.add_scalars("Training vs Validation Accuracy", {
        "training": train_correct/train_total,
        "validation": val_correct/val_total
    }, epoch)

    writer.add_scalars("Training vs Validation loss", {
        "training": trainloss/train_total,
        "validation": valloss/val_total
    }, epoch)


In [7]:
checkpoint = torch.load("model")
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
print(checkpoint["epoch"])
del checkpoint


In [8]:
#for g in optimizer.param_groups:
#    g['weight_decay'] = 0.0011


In [9]:
epochs = 1000
for t in range(380, epochs):
    for g in optimizer.param_groups:
        g['lr'] = 0.5*lr#lr*(0.995**t)
    train_loop(train_loader, model, loss_fn, optimizer, epoch=t)
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'epoch': t
        }, "model")
 

  0%|                                                 | 0/50000 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 1.12 GiB (GPU 0; 7.94 GiB total capacity; 6.78 GiB already allocated; 509.00 MiB free; 6.79 GiB reserved in total by PyTorch)